In [2]:
from scipy.sparse import csr_matrix
import itertools
import numpy as np
import mdptoolbox.mdp as mdp
import mdptoolbox.util as util
import mdptoolbox.example as example
import random



In [3]:
#model MDP flexibly, depending on size of warehouse, number of items, probabilities for items 
#model needs:
    #set of states
    #set of actions
    #transition probabilities
    #reward matrix

In [4]:
#change this input to experiment
warehouse = np.zeros(shape=(2, 2),dtype=int) #shape larger than 2x3 will take 30 plus mins to calculate transition matrix and policies
items = [(1, 0.9), (2, 0.1)]



# dont change this 
tasks = ["store", "unstore"]
greedy_items = []
for item in range(len(items)):
    greedy_items.append((items[item][0], 1/len(items)))

greedy_probabilities_only = []
for item in greedy_items:
    greedy_probabilities_only.append(item[1])    

items_only = []
for item in items:
    items_only.append(item[0])

probabilities_only = []
for item in items:
    probabilities_only.append(item[1])
#check sum of probabilities
if np.sum(probabilities_only) != 1:
    raise ValueError("The sum of item probabilities is not 1!")

In [5]:
def get_states_actions(warehouse, items):

    items_only = []
    for item in items:
        items_only.append(item[0])

    probabilities_only = []
    for item in items:
        probabilities_only.append(item[1])

    #states    
    boxstates = items_only.copy()
    boxstates.insert(0,0)
    
    nr_boxes = warehouse.shape[0]*warehouse.shape[1]
    nr_states = len(boxstates)**nr_boxes * len(tasks) * len(items)
    iterables = []

    #same set for every box
    for i in range(nr_boxes):
        iterables.append(boxstates)

    #add iterables for tasks
    iterables.append(tasks)
    iterables.append(items_only)

    states = []
    counter = 0
    for state in itertools.product(*iterables):
        counter += 1
        states.append(list(state))

    #actions
    #action is selecting the position in the warehouse for the given task
    rows = list(np.arange(warehouse.shape[0]))
    columns = list(np.arange(warehouse.shape[1]))
    nr_cells = warehouse.shape[0]*warehouse.shape[1]
    actions = []
    for action in itertools.product(*[rows, columns]):
        actions.append(list(action))
    print("states and actions done")
    return states, actions


#instanciate    
states, actions = get_states_actions(warehouse, items)

states and actions done


In [12]:
%%time

def get_tranitions_rewards(states, actions, probabilities_only=probabilities_only):
    #transitions and rewards:
    #this could be refactored and made pretty for sure!
    #matrix AxSxS', every state has probability to get to state s' by each action
    #reward depends on whether task was possible before or s==s' and distance that is determined by action
    transitions = []
    rewards = np.zeros(shape=(len(states), len(actions)))
    it_prob = 0

    for it_action in range(len(actions)):
        print("action:", it_action)
        #position in warehouse
        y,x = actions[it_action]
        # for CSR matrix
        data = []
        indptr = [0]
        indices = []
        for it_state in range(len(states)):
            #check for every state, if action is doable    
            state = states[it_state].copy()
            if state[-2] == "store":
                if state[y*warehouse.shape[0]+x] == 0:
                    #item can be stored at position
                    result = state.copy()
                    result[y*warehouse.shape[0]+x] = state[-1]                
                    #reward = 1/distance, so everyhing is positive valued and linear
                    #distance = row + column + 1 (minimal distance = 1)
                    distance = y+x+1
                    rewards[it_state, it_action] = 1/distance
                    #identify correct state in list and set probability according to item
                    for next_state in range(len(states)):
                        #iterate twice (store and unstore) over item probabilities and set transition probability accordingly
                        prob = probabilities_only[it_prob]/2
                        if (result[0:-2] == states[next_state][0:-2]):           
                            it_prob += 1
                            if it_prob == len(probabilities_only):
                                it_prob = 0
                            data.append(prob)
                            indices.append(next_state)
                    indptr.append(len(indices))

                else:
                    #reward[it_action, it_state]
                    result = state.copy()
                    #identify correct state in list and set probability according to item
                    for next_state in range(len(states)):
                        if (result == states[next_state]):           
                            data.append(1)
                            indices.append(next_state)    
                    indptr.append(len(indices))
                
            else:
                if state[y*warehouse.shape[0]+x] == state[-1]:
                    #can be unstored
                    result = state.copy()
                    result[y*warehouse.shape[0]+x] = 0
                    distance = y+x+1
                    rewards[it_state, it_action] = 1/distance
                    #identify correct state in list and set probability according to item
                    for next_state in range(len(states)):
                        #iterate twice (store and unstore) over item probabilities and set transition probability accordingly
                        prob = probabilities_only[it_prob]/2
                        if (result[0:-2] == states[next_state][0:-2]):           
                            it_prob += 1
                            if it_prob == len(probabilities_only):
                                it_prob = 0
                            data.append(prob)
                            indices.append(next_state)
                    indptr.append(len(indices))

                else:
                    #reward[it_action, it_state]
                    result = state.copy()
                    #identify correct state in list and set probability according to item
                    for next_state in range(len(states)):
                        if (result == states[next_state]):           
                            data.append(1)
                            indices.append(next_state)
                    indptr.append(len(indices))

        transition_sparse = csr_matrix((data, indices, indptr), shape=(len(states) ,len(states)), dtype=float)
        transitions.append(transition_sparse)

    # check if transitions and rewards form a valid MDP (sizewise at least)
    if util.check(transitions, rewards) is not None or len(transitions) != len(actions) or not all( i for i in [transitions[i].shape[0]==transitions[i].shape[1]==len(states) for i in range(len(transitions))]):
        raise ValueError("Size of Transition or reward matrix not correct!")
    print("transitions and rewards done")
    return transitions, rewards


#instanciate    
transitions, rewards = get_tranitions_rewards(states, actions)
g_trans, g_rewards = get_tranitions_rewards(states, actions, probabilities_only=greedy_probabilities_only)

action: 0
action: 1
action: 2
action: 3
transitions and rewards done
action: 0
action: 1
action: 2
action: 3
transitions and rewards done
Wall time: 171 ms


In [13]:
# try different MDP algorithms and get the optimal policy
    #FiniteHorizon
    #PolicyIteraiton
    #PolicyIterationModified
    #QLearning
    #RelativeValueIteration
    #ValueIteration
    #ValueIteartionGS

In [14]:
def get_policy(transitions, rewards, algorithms=["PolicyIteration", "PolicyIterationModified", "RelativeValueIteration", "ValueIteration", "ValueIteartionGS"]):
    #FiniteHorizon and QLearning seem not to converge to the optimal policy somehow....different results than other ones
    policies = {}
    discountFactor = 0.99

    #add greedy policy, algorithm doesnt matter becasue probabilitites are uniform
    greedy = mdp.PolicyIteration(g_trans, g_rewards, discountFactor, max_iter=1000)
    greedy.run()
    policies["GreedyPolicy"] = greedy.policy
    print("greedy duration", greedy.time)
    

    if "FiniteHorizon" in algorithms:
        #FiniteHorizon
        iterations = 10000
        fh = mdp.FiniteHorizon(transitions, rewards, discountFactor, N=iterations)
        fh.run()
        print("fh duration:", fh.time)
        #use iteration of policy
        policy = []
        policy_iterations = fh.policy
        for state in policy_iterations:
            policy.append(state[iterations-1])
        policies["FiniteHorizon"] = tuple(policy)
    
    if "PolicyIteration" in algorithms:
        #PolicyIteraiton
        pi = mdp.PolicyIteration(transitions, rewards, discountFactor, max_iter=1000)
        pi.run()
        policies["PolicyIteration"] = pi.policy
        print("pi duration:", pi.time)

    if "PolicyIterationModified" in algorithms:
        #PolicyIterationModified
        pim = mdp.PolicyIterationModified(transitions, rewards, discountFactor, max_iter=100000)
        pim.run()
        policies["PolicyIterationModified"] = pim.policy
        print("pim duration:", pim.time)

    if "QLearning" in algorithms:
        #QLearning not used
        ql = mdp.QLearning(transitions, rewards, discountFactor, n_iter = 10000)
        ql.run()
        policies["QLearning"] = ql.policy
        print("ql duration:", ql.time)

    if "RelativeValueIteration" in algorithms:
        #RelativeValueIteration
        rvi = mdp.RelativeValueIteration(transitions, rewards, max_iter=20000)
        rvi.run()
        policies["RelativeValueIteration"] = rvi.policy
        print("rvi duration:", rvi.time)

    if "ValueIteration" in algorithms:
        #ValueIteration
        vi = mdp.ValueIteration(transitions, rewards, discountFactor, max_iter=2000000)
        vi.run()
        policies["ValueIteration"] = vi.policy
        print("vi duration:", vi.time)

    if "ValueIteartionGS" in algorithms:
        #ValueIteartionGS
        vigs = mdp.ValueIterationGS(transitions, rewards, discountFactor, max_iter=1000000)
        vigs.run()
        policies["ValueIteartionGS"] = vigs.policy
        print("vigs duration:", vigs.time)
    
    print("policies done")
    return policies

#instanciate
policies = get_policy(transitions, rewards)

greedy duration 4.903049945831299
pi duration: 5.309864044189453
pim duration: 0.005013942718505859
rvi duration: 0.002005338668823242
vi duration: 0.004012584686279297
vigs duration: 3.3375635147094727
policies done


In [15]:
def get_rec_action(policy, state):
    if len(policy)!=len(states):
        return "Policy does not match number of states!"
    #give state and get action(position based on chosen policy)
    for it_state in range(len(states)):
        #find position of state to search policy at this posiiton 
        if states[it_state]==state:
            return actions[policy[it_state]]
    return "Input state is invalid, check shape and items!"

In [16]:
#to display options to test out states
print("Warehouse input length:", warehouse.shape[0]*warehouse.shape[1])
print("Possible actions:", actions)
print("Possible tasks: store, unstore")
print("Possible items:", items)

Warehouse input length: 4
Possible actions: [[0, 0], [0, 1], [1, 0], [1, 1]]
Possible tasks: store, unstore
Possible items: [(1, 0.9), (2, 0.1)]


In [19]:
#experiemnt here with this state and policy
test_state = [0, 0, 0, 0,  'store', 2]


for key in policies:
    print("For state", test_state,"policy", key,  get_rec_action(policies[key], test_state))

For state [0, 0, 0, 0, 'store', 2] policy GreedyPolicy [0, 0]
For state [0, 0, 0, 0, 'store', 2] policy PolicyIteration [1, 0]
For state [0, 0, 0, 0, 'store', 2] policy PolicyIterationModified [0, 1]
For state [0, 0, 0, 0, 'store', 2] policy RelativeValueIteration [0, 1]
For state [0, 0, 0, 0, 'store', 2] policy ValueIteration [0, 1]
For state [0, 0, 0, 0, 'store', 2] policy ValueIteartionGS [0, 1]


In [ ]:
# compare greedy policy to the optimal policy by simulating a number of of connected warehouse states
    # we can get the greedy policy by searching the optimal policy for items with a uniform probability distribution
    # as this always prefers the closest free position/item for all items

In [23]:
def simulate(policy, iterations=100, randomstart=False, printer=False):
    reward=[]

    # either start with random state or empty warehouse with random storing task
    if randomstart:
        state = (random.choices(states, k=1)[0]).copy()
    else:
        warhouse_state = warehouse.tolist()[0]
        warhouse_state.append("store")
        warhouse_state.append(random.choices(items_only, tuple(probabilities_only), k=1)[0])
        state = warhouse_state.copy()

    for i in range(iterations):
        if printer:
            print()
            print("iteration", i+1)
        # sanity checking states
        # intervene for states that result in loops (impossible tasks)
        # unstore item that is not in warehouse
        if state[-2] == "unstore" and not (np.array(state[0:-2])==state[-1]).any():
            state[-2] = "store"
            if printer:
                print("changed to store", state)
        # store item in full warehouse
        if state [-2] == "store" and (np.array(state[0:-2])!=0).all():
            state[-2]="unstore"
            if printer:
                print("changed to unstore", state)
            # can neither unstore nor store then change item
            if (np.array(state[0:-2])!=state[-1]).all():
                state[-1] = random.choices(items_only, tuple(probabilities_only), k=1)[0]
                if printer:
                    print("changed item", state)
        
        
        #get recommended action based on policy and current state  
        action = get_rec_action(policy, state)
        if printer:
            print("start", state)
            print("action", action)         
        
        #find index of action and state
        for it_action in range(len(actions)):
            if action == actions[it_action]:
                break       
        for it_state in range(len(states)):
            if state == states[it_state]:
                break
        
        # add reward based on state and action
        reward.append(rewards[it_state][it_action])
        if printer:
            print("reward", rewards[it_state][it_action])
        
        # find possible next states(indices) from transition matrix
        possible_next = list(np.where((transitions[it_action][it_state]).toarray() != 0)[1])
        probabilites_next = []
        for i in possible_next:
            probabilites_next.append(transitions[it_action].toarray()[it_state][i])
        if printer:
            print("possible next", possible_next)
            print("probabilities next", probabilites_next)

        # pick next state based on assigned probability 
        it_state = random.choices(possible_next, probabilites_next, k=1)[0]
        state = states[it_state].copy()
        if printer:
            print("result",it_state, state)
        
        
    #return average reward
    return np.sum(reward)/iterations

In [25]:
simulate(policies["GreedyPolicy"], iterations=5, randomstart=True, printer=True)


iteration 1
changed to unstore [1, 2, 1, 1, 'unstore', 2]
start [1, 2, 1, 1, 'unstore', 2]
action [0, 1]
reward 0.5
possible next [124, 125, 126, 127]
probabilities next [0.45, 0.05, 0.45, 0.05]
result 127 [1, 0, 1, 1, 'unstore', 2]

iteration 2
changed to store [1, 0, 1, 1, 'store', 2]
start [1, 0, 1, 1, 'store', 2]
action [0, 1]
reward 0.5
possible next [196, 197, 198, 199]
probabilities next [0.45, 0.05, 0.45, 0.05]
result 196 [1, 2, 1, 1, 'store', 1]

iteration 3
changed to unstore [1, 2, 1, 1, 'unstore', 1]
start [1, 2, 1, 1, 'unstore', 1]
action [0, 0]
reward 1.0
possible next [88, 89, 90, 91]
probabilities next [0.45, 0.05, 0.45, 0.05]
result 90 [0, 2, 1, 1, 'unstore', 1]

iteration 4
start [0, 2, 1, 1, 'unstore', 1]
action [1, 0]
reward 0.5
possible next [76, 77, 78, 79]
probabilities next [0.45, 0.05, 0.45, 0.05]
result 76 [0, 2, 0, 1, 'store', 1]

iteration 5
start [0, 2, 0, 1, 'store', 1]
action [0, 0]
reward 1.0
possible next [184, 185, 186, 187]
probabilities next [0.45, 

0.7

In [36]:
%%time

# compare greedy and optimized policy for a numer of simulations (if iterations is too short, difference will be hard to see if porbability of some items is too low)
# larger number of simulations achieves greater stability
greedy=[]
optimized=[]
for i in range(50):
    optimized.append(simulate(policies["PolicyIteration"], iterations=1000))
    greedy.append(simulate(policies["GreedyPolicy"], iterations=1000))

print("Optimized policy mean reward:", np.mean(optimized))
print("Greedy policy mean reward:", np.mean(greedy))

Optimized policy mean reward: 0.7463975
Greedy policy mean reward: 0.6978208333333333
Wall time: 2min 16s


In [ ]:
# the end